# **Import Modules**

In [49]:
import sys

import os

sys.path.insert(0, '../input/deeplearning-utils')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [57]:
from utils.external.common import OS

from utils.tf.ops import io

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

from tqdm import tqdm

In [6]:
from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets

In [11]:
import tensorflow as tf

from tensorflow.keras import backend, optimizers

from tensorflow.keras.layers import *

from tensorflow.keras.models import Model

# from tensorflow.keras.applications import *

import tensorflow_addons as tfa

# **Config**

In [27]:
class Config:
    
    SEED = 2053
    
    IMAGE_SIZE = (224, 224)

    TRAIN_BATCH_SIZE = 32
    VAL_BATCH_SIZE = 32
    
    TEST_BATCH_SIZE = 32
    
    SPLIT = 10

In [7]:
try:
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    
    print('Running on TPU ', tpu.master())
    
except ValueError:
  
    tpu = None

if tpu is not None:
    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy(tpu)

else:
    
    strategy = tf.distribute.get_strategy()


print("REPLICAS: ", strategy.num_replicas_in_sync)

In [9]:
# user_secrets = UserSecretsClient()
# user_credential = user_secrets.get_gcloud_credential()

# user_secrets.set_tensorflow_credential(user_credential)

# GCS_DS_PATH = KaggleDatasets().get_gcs_path('dont-stop-until-you-drop')

# **Load Data**

In [12]:
src_dir = '../input/ci-sc22-places-and-scene-recognition/'

In [67]:
train_meta = pd.read_csv('../input/ci-sc22-places-and-scene-recognition/train.csv')

train_meta['image'] = src_dir + 'train_images/train_images/' + train_meta['image_name']

train_meta = train_meta.sample(frac=1, random_state=Config.SEED).reset_index(drop=True)

In [68]:
print(np.unique(train_meta.label, return_counts=True))

print(train_meta.iloc[0].image)

(array([0, 1, 2, 3, 4, 5]), array([2102, 2196, 2366, 2430, 2227, 2306]))
../input/ci-sc22-places-and-scene-recognition/train_images/train_images/5634.jpg


# **Write TFRecords**

In [73]:
# n_records = 50

# writer = io.TfRecordWriter(size=Config.IMAGE_SIZE, n_records=n_records, image_ext='.jpg', quality=100, 
#                            failure_ignore=False, cache_warnings=False)

In [74]:
# write_train = True

# if write_train:

#     if not OS.dir_exists('./train/'):

#         os.mkdir('./train/')

#     writer.from_dataframe(dataframe=train_meta, dtypes=dtype, image_key='image', pref_fname='train',
#                           from_dir=None, to_dir='./train', has_ext=True)

In [76]:
# !zip -r train.zip ./train/*
# !rm -r ./train